In [1]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Helpers
import sys
from datetime import datetime
import pickle

from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,log_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut,KFold,StratifiedKFold
from sklearn.decomposition import PCA,KernelPCA
import lightgbm as lgb

import sys
sys.path.insert(0,'../../') 
from helpers import *
from MAP_estimator import MAP_estimator

# Algorithm
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'

In [2]:
with open(data_path+'train_dict.pickle','rb') as f:
                count_dict = pickle.load(f) 

In [9]:
with open(data_path+'file_names(train).pickle','rb') as f:
                file_name = pickle.load(f) 

In [13]:
sn = [n.replace('.tsv','') for n in file_name]

In [15]:
la = pd.read_csv(data_path+'train_Y.csv')

In [63]:
with open(data_path+'inc_p_values.pkl','rb') as f:
                inc_p_values = pickle.load(f) 

In [34]:
TCRs_inc[inc_p_values.columns.values[0]]

[2, 0]

In [60]:
with open(data_path+'TCRs_incidences_dict.pkl','rb') as f:
                TCRs_inc = pickle.load(f) 

In [64]:
inc_p_values

TCRBV01                                                             \
        TCRBV01-01                                                              
                01                                                              
         CTFQETQYF CTHGGHSLPTPSNQPQHF CTRGQGGTEAFF   CTSGEQYF CTSGGLAESTDTQYF   
           TCRBJ02            TCRBJ01      TCRBJ01    TCRBJ02         TCRBJ02   
        TCRBJ02-05         TCRBJ01-05   TCRBJ01-01 TCRBJ02-07      TCRBJ02-03   
                01                 01           01         01              01   
CMV+      2.000000                0.0          0.0    1.00000         1.00000   
CMV-      0.000000                2.0          2.0    1.00000         1.00000   
p_value   0.202886                1.0          1.0    0.69883         0.69883   

                                                                        \
                                                                         
                                                                         
        CTSGPSNQPQHF CTSGPYEQYF  CTSGYEQYF CTSILTRYNSNQPQHF CTSNPTEAFF   
             TCRBJ01    TCRBJ02    TCRBJ02          TCRBJ01    TCRBJ01   
          TCRBJ01-05 TCRBJ02-07 TCRBJ02-07       TCRBJ01-05 TCRBJ01-01   
                  01         01         01               01         01   
CMV+        2.000000    1.00000        0.0              0.0    1.00000   
CMV-        0.000000    1.00000        3.0              2.0    1.00000   
p_value     0.202886    0.69883        1.0              1.0    0.69883   

             ...                       null                       \
             ...                 unresolved                        
             ...                       null                        
             ...       CASSSSGLAGGRSSYNEQFF CASSSSPGLAGGSSYNEQFF   
             ...                    TCRBJ02              TCRBJ02   
             ...                 TCRBJ02-01           TCRBJ02-01   
             ...                         01                   01   
CMV+         ...                    1.00000              1.00000   
CMV-         ...                    1.00000              1.00000   
p_value      ...                    0.69883              0.69883   

                                                                    \
                                                                     
                                                                     
        CASSSTGAGNQPQHF CASSSYNLRGGGRGGRNEQFF CASSTGLAGGLSSGANVLTF   
                TCRBJ01               TCRBJ02              TCRBJ02   
             TCRBJ01-05            TCRBJ02-01           TCRBJ02-06   
                     01                    01                   01   
CMV+           2.000000                   0.0             2.000000   
CMV-           0.000000                   2.0             0.000000   
p_value        0.202886                   1.0             0.202886   

                                                                    \
                                                                     
                                                                     
        CASSTPGLAGGSSSYNEQFF CASSVGGRGKNTEAFF CASSYPGLAGGSSSYNEQFF   
                     TCRBJ02          TCRBJ01              TCRBJ02   
                  TCRBJ02-01       TCRBJ01-01           TCRBJ02-01   
                          01               01                   01   
CMV+                 1.00000              0.0             2.000000   
CMV-                 1.00000              2.0             0.000000   
p_value              0.69883              1.0             0.202886   

                                        
                                        
                                        
        CASTRTESSYNEQFF CATGTGDSNQPQHF  
                TCRBJ02        TCRBJ01  
             TCRBJ02-01     TCRBJ01-05  
                     01             01  
CMV+                0.0        1.00000  
CMV-                2.0        1.00000  
p_value

In [66]:
inc_pos = pd.read_pickle(data_path+'TCRs_ranking_ind_pos.pkl')

In [67]:
inc_pos

array([5475728, 7756469, 1797827, ..., 4396306, 1092951,  978170])

In [44]:
inc_p_values.iloc[:,inc_pos.tolist()[:100]]

TCRBV09         TCRBV19         TCRBV05       TCRBV07  \
           TCRBV09-01      TCRBV19-01      TCRBV05-01    TCRBV07-02   
                 null            null              01            01   
        CASSGQGAYEQYF CASSIGPLEHNEQFF CASSPDRVGQETQYF CASSLEAEYEQYF   
              TCRBJ02         TCRBJ02         TCRBJ02       TCRBJ02   
           TCRBJ02-07      TCRBJ02-01      TCRBJ02-05    TCRBJ02-07   
                   01              01              01            01   
CMV+     6.100000e+01    3.000000e+01    3.300000e+01  3.000000e+01   
CMV-     1.200000e+01    0.000000e+00    1.000000e+00  1.000000e+00   
p_value  7.958014e-13    1.751455e-11    2.664239e-11  3.202211e-10   

              TCRBV28       TCRBV24         TCRBV05       TCRBV07  \
           TCRBV28-01    unresolved      TCRBV05-06    TCRBV07-06   
                   01          null              01            01   
        CASSIEGNQPQHF   CATSDGDEQFF CASSLVAGGRETQYF CASSRGRQETQYF   
              TCRBJ01       TCRBJ02         TCRBJ02       TCRBJ02   
           TCRBJ01-05    TCRBJ02-01      TCRBJ02-05    TCRBJ02-05   
                   01            01              01            01   
CMV+     2.600000e+01  4.100000e+01    5.000000e+01  3.700000e+01   
CMV-     0.000000e+00  6.000000e+00    1.100000e+01  5.000000e+00   
p_value  5.306200e-10  5.866204e-10    6.102922e-10  2.560004e-09   

                TCRBV09       TCRBV04      ...             TCRBV02  \
             TCRBV09-01    TCRBV04-03      ...          TCRBV02-01   
                   null            01      ...                  01   
        CASSAGQGVTYEQYF CASSPQRNTEAFF      ...      CASRDWDYTDTQYF   
                TCRBJ02       TCRBJ01      ...             TCRBJ02   
             TCRBJ02-07    TCRBJ01-01      ...          TCRBJ02-03   
                     01            01      ...                  01   
CMV+       2.400000e+01  2.900000e+01      ...           13.000000   
CMV-       0.000000e+00  2.000000e+00      ...            0.000000   
p_value    2.882701e-09  6.654355e-09      ...            0.000027   

               TCRBV06       TCRBV04        TCRBV11        TCRBV05  \
            TCRBV06-05    TCRBV04-03     TCRBV11-02     TCRBV05-01   
                    01            01             02             01   
        CASRGQGWDEKLFF CASSPHRNTEAFF CASSLGDRPDTQYF CASSLEGQGFGYTF   
               TCRBJ01       TCRBJ01        TCRBJ02        TCRBJ01   
            TCRBJ01-04    TCRBJ01-01     TCRBJ02-03     TCRBJ01-02   
                    01            01             01             01   
CMV+         13.000000     13.000000      13.000000      13.000000   
CMV-          0.000000      0.000000       0.000000       0.000000   
p_value       0.000027      0.000027       0.000027       0.000027   

                                              TCRBV03        TCRBV25  \
             TCRBV05-06      TCRBV05-01    unresolved     TCRBV25-01   
                     01              01          null             01   
        CASSLVAAGRETQYF CASSSDRVGQETQYF CASSQVETDTQYF CASSWDRDNSPLHF   
                TCRBJ02         TCRBJ02       TCRBJ02        TCRBJ01   
             TCRBJ02-05      TCRBJ02-05    TCRBJ02-03     TCRBJ01-06   
                     01              01            01             01   
CMV+          13.000000       13.000000     13.000000      13.000000   
CMV-           0.000000        0.000000      0.000000       0.000000   
p_value        0.000027        0.000027      0.000027       0.000027   

              TCRBV05  
           TCRBV05-06  
                   01  
        CASSSGQVYGYTF  
              TCRBJ01  
           TCRBJ01-02  
                   01  
CMV+        22.000000  
CMV-         4.000000  
p_value      0.000028  

[3 rows x 100 columns]

In [46]:
del inc_pos

In [49]:
with open(data_path+'labeled_files_ind.pkl','rb') as f:
                labelled_f = pickle.load(f) 

In [51]:
ss = np.take(sn,labelled_f)

In [53]:
ss.tolist() == la['sample_name'].tolist()

True

In [90]:
from scipy.sparse import load_npz
bin = load_npz(data_path+'X_sparse/'+'train_freq.npz')

In [91]:
np.sum(bin[:,:10].todense(),axis=0)

matrix([[1.08480453e-05, 1.11460686e-05, 9.05990601e-06, 1.26957893e-05,
         1.04904175e-05, 7.51018524e-06, 1.28149986e-05, 1.85370445e-05,
         1.59144402e-05, 8.34465027e-06]], dtype=float32)

In [70]:
test = bin[:,inc_pos.tolist()[:10]].todense()

In [79]:
s = la['sample_name'][0]

In [82]:
bin[la.index[la['sample_name']==s].tolist(),:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint16)

In [71]:
test

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0]], dtype=uint16)

In [72]:
pos = la.index[la['phenotype_status'] == 1].tolist()
neg = la.index[la['phenotype_status'] == 0].tolist()

In [73]:
test_pos = test[pos,:]
test_neg = test[neg,:]

In [75]:
test_pos.shape

(289, 10)

In [76]:
np.sum(test_pos,axis=0)

matrix([[61, 30, 33, 30, 26, 41, 50, 37, 24, 29]], dtype=uint64)

In [77]:
inc_p_values.iloc[:,inc_pos.tolist()[:10]]

,TCRBV09,TCRBV19,TCRBV05,TCRBV07,TCRBV28,TCRBV24,TCRBV05,TCRBV07,TCRBV09,TCRBV04
,TCRBV09-01,TCRBV19-01,TCRBV05-01,TCRBV07-02,TCRBV28-01,unresolved,TCRBV05-06,TCRBV07-06,TCRBV09-01,TCRBV04-03
,null,null,01,01,01,null,01,01,null,01
,CASSGQGAYEQYF,CASSIGPLEHNEQFF,CASSPDRVGQETQYF,CASSLEAEYEQYF,CASSIEGNQPQHF,CATSDGDEQFF,CASSLVAGGRETQYF,CASSRGRQETQYF,CASSAGQGVTYEQYF,CASSPQRNTEAFF
,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ01,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ01
,TCRBJ02-07,TCRBJ02-01,TCRBJ02-05,TCRBJ02-07,TCRBJ01-05,TCRBJ02-01,TCRBJ02-05,TCRBJ02-05,TCRBJ02-07,TCRBJ01-01
,01,01,01,01,01,01,01,01,01,01
CMV+,6.100000e+01,3.000000e+01,3.300000e+01,3.000000e+01,2.600000e+01,4.100000e+01,5.000000e+01,3.700000e+01,2.400000e+01,2.900000e+01
CMV-,1.200000e+01,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,1.100000e+01,5.000000e+00,0.000000e+00,2.000000e+00
p_value,7.958014e-13,1.751455e-11,2.664239e-11,3.202211e-10,5.306200e-10,5.866204e-10,6.102922e-10,2.560004e-09,2.882701e-09,6.654355e-09


In [78]:
np.sum(test_neg,axis=0)

matrix([[12,  0,  1,  1,  0,  6, 11,  5,  0,  2]], dtype=uint64)